### Basic RAG retriever using Langchain and Anthropic model

In [1]:
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_anthropic import ChatAnthropic
from dotenv import load_dotenv
import os
load_dotenv()
os.environ["LANGCHAIN_API_KEY"]= os.getenv("LANGCHAIN_API_KEY")
llm = ChatAnthropic(model="claude-3-5-sonnet-20240620")


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [13]:
vectorestore= Chroma(embedding_function=OllamaEmbeddings(model='nomic-embed-text'),)

In [16]:

retriever = vectorestore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("who is madame karenina")

'Based on the provided context, Madame Karenina appears to be a character in the story who is the subject of gossip and social controversy. She is mentioned as being seen with Alexey Vronsky and is involved in some kind of social incident with Madame Kartasova at what seems to be a theater or opera. However, the context does not provide enough information to definitively state who Madame Karenina is or her full role in the story.'